In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'personal-info-txt:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5651646%2F9328273%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240909%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240909T072234Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc2a269b9e3a6377e6aac3fbcf5889a6e885fea9fda867e1881704b1bb57d71b96c7781897029c7fbebca8e8885df963ea8e4f486f08ac781408838d6ea952d9020b7fc94e4fc2ceae93aed95c46ffef17c6b75ff494d9491f99cbdbbb78defe2cb8bc3f4b9bc0d5e5877502fb1b7c8036b1314328eef55e8fdc5b6a58db5bc75a09e13c3de9ec7aa28fb5e16cb778359c41aeb79b9fbb5a8e56720081a1186d1f4b5f07a32e97fd92bb45ea2ef146757db62197499ae3a8d65b9047d1e1151ac9d75664bcc3d56d5261a94a9fad71e38aef7a69b8c2ca7ac1199ff5008848b6ed19bab06b974bce2a187a9494a3de49545a181b54eda9cc7fb21b5c98d40eaa5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2246 bytes downloaded
Downloaded and uncompressed: personal-info-txt
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
'''install these if not installed already

if you want to run this on your local system using ollama you can download the packages
needed for it at https://ollama.com/download
after installation setup you have to open command prompt or terminal and type the "command ollama pull llama3"
for llama3 if you want other LLMs there are a bunch of others as well at https://github.com/ollama/ollama/blob/main/README.md

After you install you can add a web interface to the chatbot if you want to use an open source project API to link to ollama
you can find it here https://github.com/open-webui/open-webui , i did not use this but you can use it if you like.

then come to VS code and pip install these packages using terminal

"pip install langchain langchain_ollama ollama" then run this code below while adding personal_info.txt file
 in the code directory and you are all set.'''

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

template = '''

Answer the question below
here is the conversation history : {context}

question : {Question}

Answer :
'''

model = OllamaLLM(model="llama3")
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

def load_file_content(filename):
    """Load the content of the specified text file."""
    with open(filename, 'r') as file:
        return file.read()

def handle_convo(file_content):
    context = file_content
    print("\nwelcome to my chatbot using LLama and langchain, type 'exit' to exit out of chatbot\n\nYou can use this chatbot to know info about sriker joshi\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Exiting the chatbot. Context will be reset.")
            context = file_content  # Reset context to initial content after exiting
            break

        result = chain.invoke({"context": context, "Question" : user_input})
        print("chatbot: ", result)
        context += f"\nUser: {user_input}\n chatbot: {result}"

if __name__=="__main__":

    file_content = load_file_content("/kaggle/input/personal-info-txt/personal_info.txt")

    handle_convo(file_content)
